# Data Import



In [42]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json

In [43]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

epc_path = config['DEFAULT']['epc_path']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']
epc_output_fname = config['DEFAULT']['epc_output_fname']
processing_path = config['DEFAULT']['processing_path']

In [44]:
print(config_path)

/home/jovyan/work/01_Code/01_EPC


In [45]:
dtype_dict = {'INSPECTION_DATE':'str','LODGEMENT_DATE':'str'}
epc_wales = pd.read_csv(os.path.join(processing_path,'output.csv'),header=0,delimiter=',',dtype=dtype_dict,parse_dates=['INSPECTION_DATE'])

FileNotFoundError: [Errno 2] No such file or directory: '../../../output/output.csv'

In [46]:
epc_wales['CURRENT_ENERGY_RATING'].value_counts()

NameError: name 'epc_wales' is not defined

In [47]:
len(epc_wales)

NameError: name 'epc_wales' is not defined

### Import EPC data and concatenate to a single dataframe 

In [48]:
#Get a list of all the files in  the directory
epc_filenames = os.listdir(epc_path) #350 elements
#epc_filenames = epc_filenames[:10]

dtype_dict = {'INSPECTION_DATE':'str','LODGEMENT_DATE':'str'}

tmp_list = [] #List of dataframes
df_count = 0  #The count of imported dataframes

for fname in epc_filenames:
    if fname.endswith(epc_fname_suffix):
        print("Reading in dataframe %i: %s" % (df_count, fname))
        tmp = pd.read_csv(os.path.join(epc_path,fname),
                          header = 0,
                          #nrows = 2,
                          delimiter = ',',
                          dtype = dtype_dict,
                          parse_dates = ['INSPECTION_DATE','LODGEMENT_DATE'])
        #Extract the region from the filename and add to the dataframe
        tmp['region'] = fname.split(' certificates.csv')[0]
        #Append the imported dataframe to the list of dataframes
        tmp_list.append(tmp)
        #Increment the counter
        df_count += 1

#TODO columns 3, 15, 51, 59 and 60 have mixed data types consider specifying dtypes

#Concatenate the list of dataframes to a single dataframe
epc_data = pd.concat(tmp_list,ignore_index = True)

Reading in dataframe 0: certificates.csv


### Export Data

In [49]:
#Sort
epc_data.sort_values(['region','LMK_KEY'],inplace = True)

#Export
epc_data.to_csv(os.path.join(processing_path,epc_output_fname) + epc_fname_suffix,index = False)